In [1]:
from packages import *
from nltk.tokenize import word_tokenize
import re
import unicodedata
import pickle
import concurrent.futures
import tensorflow as tf
%load_ext autoreload
%autoreload 2

In [2]:
glove_dim = 50

In [3]:
tokens_to_keep = set( load_pickle(os.path.join(GLOVE_DIR, 'glove-{}D.pkl'.format(glove_dim))).keys())

In [4]:
data = load_data({'review': ['text']})['review']['text']

In [7]:
review[12]

"Wow. So surprised at the one and two star reviews!  We started with the most tender calamari. Although the marinara sauce was a bit bland, but a touch of salt made it just right. My husband had the veal with peppers and said it was so delicious and tender. The mashed potatoes were perfect. I had the salmon Diablo which was also delicious. Our salad was beautiful! Dressing was served on the salad and it was a nice amount. We ended our delicious meal with a piece of tiramisu. Our server Matt was right on!! Very pleasant and knowledgeable about the menu. Our appetizer, salad and entrees were timed perfectly. I love salad and did not mind that my entree was served while I was still eating it! No problem it let my dinner cool to just the right temp for me to eat it comfortably. \nI wonder sometimes if people just don't appreciate relaxing and taking time to eat a wonderful and beautifully prepared meal.  A wonderful atmosphere. So relaxing. The chairs are super comfortable too!!! We will c

In [5]:
# token_lengths = [(len(token), token) for token in tokens_to_keep]
# token_lengths.sort()
# token_lengths = token_lengths[::-1]

In [6]:
max_token_length = 16
review_length = 300
dtype = '|S{}'.format(max_token_length) #itemsize = 16

In [7]:
NULL_KEY in tokens_to_keep, UNK_KEY in tokens_to_keep

(False, False)

In [8]:
def clean_token(t):
    t = unicodedata.normalize('NFKD', t).encode('ascii', 'ignore').decode() #sometimes len() increases after normalization
    if len(t) > max_token_length:
        return UNK_KEY
    t = t.lower()
    if t not in tokens_to_keep:
        return UNK_KEY
    m = re.match('^[^\w\'](\w+).*', t)
    if m is not None:
        t = m.group(1)
    return t

def upsample(tokens):
    # upsampling strategy is to pad with NULL_KEY's after the last token.
    shortage = review_length - len(tokens)
    tokens += [NULL_KEY] * shortage
    return tokens

def downsample(tokens):
    # dowsampling strategy is to remove all UNK_KEY's before touching other tokens
    excess = len(tokens) - review_length
    downsampled_tokens = []
    for i, t in enumerate(tokens):
        if excess == 0:
            downsampled_tokens += tokens[i:]
            return downsampled_tokens
        if t == UNK_KEY:
            excess-=1
        else:
            downsampled_tokens.append(t)
    if excess == 0:
        downsampled_tokens += tokens[i+1:]
        return downsampled_tokens       
    # if we reach here, it means previous attempt didn't get rid of all of the excess, and we will truncate both ends
    beg = excess // 2
    end = excess - beg
    downsampled_tokens = downsampled_tokens[beg:-end]
    return downsampled_tokens

def tokenize(review):
    tokens = [clean_token(t) for t in word_tokenize(review)]
    if len(tokens) < review_length:
        tokens = upsample(tokens)
    elif len(tokens) > review_length:
        tokens = downsample(tokens)
    return np.array(tokens, dtype=dtype)

def preprocess(review):
    return tokenize(review)

In [ ]:
data_preprocessed = []
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    data_preprocessed = executor.map(preprocess, data)
data_preprocessed = [i for i in data_preprocessed]

In [33]:
N = len(data_preprocessed)
splits = 100
per_split = N // splits

In [ ]:
for s in range(splits):
    x = np.array(data_preprocessed[s * per_split: (s+1) * per_split])
    np.save(os.path.join(DATASET_DIR, 'preprocessed', 'review-text-{:02d}.npy'.format(s)), x,
            allow_pickle=False)

In [78]:
def serialize_example(review):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {
      'review': _bytes_feature(review),
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [79]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [95]:
def tf_serialize_example(review):
    tf_string = tf.py_function(
    serialize_example,
    review,  # pass these args to the above function.
    tf.string)      # the return type is `tf.string`.
    return tf.reshape(tf_string, ()) # The result is a scalar

In [40]:
writer = tf.io.TFRecordWriter(os.path.join(DATASET_DIR, 'preprocessed', 'review-text_preprocessed-1.tf'))

In [96]:
ds = tf.data.Dataset.from_tensor_slices(x)

In [85]:
y =ds.take(1)

In [89]:
for i in y:
    break

In [106]:
import glob

In [129]:
'{:03d}'.format(1)

'001'

In [97]:
tf.data.Dataset.map(ds, tf_serialize_example)

TypeError: Expected list for 'input' argument to 'EagerPyFunc' Op, not Tensor("args_0:0", shape=(300,), dtype=string).